<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from typing import List
from datetime import datetime
from random import randint
from dateutil.relativedelta import *


In [3]:
    factionName: List[str] = []
    systemName: List[str] = []
    population: List[int] = []
    influence: List[float] = []
    updated: List[datetime] = []

In [6]:
factionName: List[str] = []
systemName: List[str] = []
population: List[int] = []
influence: List[float] = []
updated: List[datetime] = []
    
randStrs = ['foo', 'bar', 'guh']

def randStr() -> str:
  global randStrs
  return randStrs[randint(0, len(randStrs))]

def seqStr(i: int) -> str:
  global randStrs
  return randStrs[i % len(randStrs)]

start_date = datetime.strptime("2020-06-01", "%Y-%m-%d")


for fac in ['foo','bar','guh']:
  upd = start_date
  for sys in ['sol','celeano']:
    for i in range(0,5):
      factionName.append( fac)
      systemName.append( sys)
      population.append(10000)
      influence.append(.33)

      day = randint(2,4)
      upd = upd + relativedelta(hours=day*8)
      updated.append(upd)


data = { 'systemName' : systemName,
         'factionName' : factionName,
         'population' : population,
         'influence'  : influence,
         'updated' : updated
        }
df = pd.DataFrame(data = data)


In [47]:
# prepare data

df['updated'] = df.updated.dt.round("D") 
df = df.sort_values(by=['systemName', 'factionName','updated'])

# this drops out all data that doesn't have at least 2 days in a row
systems = df['systemName'].unique().tolist()
holder = pd.DataFrame()

for sys in systems:
  print(sys)
  sysSlice = df[ df['systemName'] ==sys]
  facs = sysSlice['factionName'].unique().tolist()
  for fac in facs:
    print("\t"+ fac)
    facSlice = sysSlice[ sysSlice['factionName'] == fac].reset_index()
    facSlice['diff1'] = facSlice.updated.diff(periods=1).dt.days.abs()
    facSlice['diff2'] = facSlice.updated.diff(periods=-1).dt.days.abs()
    facSlice = facSlice.fillna(999.0) 
    facSlice['diffz'] = facSlice[['diff1','diff2']].min(axis=1)
    facSlice = facSlice[ facSlice['diffz'] <=1.0]
    facSlice = facSlice.drop(['diff1','diff2','diffz'],  axis=1)
    holder = holder.append(facSlice.copy(), ignore_index=True)


holder

celeano
	bar
	foo
	guh
sol
	bar
	foo
	guh


,index,systemName,factionName,population,influence,updated
0,15,celeano,bar,10000,0.33,2020-06-06
1,16,celeano,bar,10000,0.33,2020-06-07
2,17,celeano,bar,10000,0.33,2020-06-08
3,18,celeano,bar,10000,0.33,2020-06-09
4,5,celeano,foo,10000,0.33,2020-06-07
5,6,celeano,foo,10000,0.33,2020-06-08
6,7,celeano,foo,10000,0.33,2020-06-09
7,8,celeano,foo,10000,0.33,2020-06-10
8,9,celeano,foo,10000,0.33,2020-06-11
9,25,celeano,guh,10000,0.33,2020-06-08


In [32]:
def minAbs(x1: float, x2: float) -> float:
  return min ( abs(x1), abs(x2))